# Exercise 3.2 (Moon Ephemerides)

In [13]:
using NBInclude
using LinearAlgebra,Plots

In [14]:
@nbinclude("./SAT_Time.ipynb")
@nbinclude("./SAT_Kepler.ipynb")
@nbinclude("./SAT_Force.ipynb")
@nbinclude("./SAT_Const.ipynb")

4.56e-6

- Satellite Orbits (pag.113)

The Moon ephemeris is to be computed and
evaluated. To this end the geocentric Cartesian position coordinates of the Moon are
to be computed from 2006/03/14 00:00 to 2006/03/18 00:00 (Terrestrial Time) in
steps of one day. A comparison of low-precision analytic lunar coordinates is to be
made with positions as derived from the Chebyshev coefficients of JPL’s DE405. A
listing of the 39 Chebyshev coefficients (13 per coordinate) is given below, which
covers the DE405 subinterval size of 4 days.

## 3.3- Sun and Moon

Sample Chebyshev coefficients from JPL DE405 for geocentric lunar 
coordinates in the ITRF(EME2000) frame valid from 2006/03/14 TDB 
to 2006/03/18 TDB 

In [3]:
N_coeff = 13          # Number of coefficients 
t1 = 53808.0          # MJD at start of interval 
t2 = 53812.0          # MJD at end   of interval 
Cx_moon=              # Coefficients for x-coordinate
[-0.383089044877016277e+06, 0.218158411754834669e+05, 0.179067292901463843e+05,
 -0.836928063411765777e+02,-0.628266733052023696e+02,-0.459274434235101225e+00,
  0.491167202819885532e-01, 0.770804039287614762e-03,-0.125935992206166816e-03,
  0.500271026610991370e-05, 0.107044869185752331e-05, 0.172472464343636242e-08,
 -0.269667589576924680e-08]
Cy_moon=              #Coefficients for y-coordinate
[-0.379891721705081436e+05,-0.143611643157166138e+06, 0.187126702787245881e+04, 
  0.112734362473135207e+04, 0.932891213817359177e+00,-0.191932684130578513e+01, 
 -0.266517663331897990e-01, 0.104558913448630337e-02,-0.359077689123857890e-04, 
 -0.123405162037249834e-04, 0.180479239596339495e-06, 0.525522632333670539e-07, 
  0.543313967008773005e-09]                                              
Cz_moon=              #Coefficients for z-coordinate
[-0.178496690739133737e+05,-0.788257550331743259e+05, 0.880684692614081882e+03, 
  0.618395886330471512e+03, 0.103331218594995988e+01,-0.104949867328178592e+01,
 -0.150337371962561087e-01, 0.569056416308259317e-03,-0.186297523286550968e-04,
 -0.680012420653791955e-05, 0.902057208454410917e-07, 0.287891446432139173e-07, 
  0.319822827699973363e-09];

#Cheb3D(MoonCheb,N_coeff,t1,t2,Cx_moon,Cy_moon,Cz_moon)

### Moon position from DE405

In [11]:
function MoonCheb( r_Moon, t,
                   N,Ta,Tb,Cx,Cy,Cz)
"""
 Evaluation of the Chebyshev approximation of a three-dimensional vector
"""    
  if ( (t<Ta) || (Tb<t) ) 
    println("ERROR: Time out of range in Cheb3D::Value")
    exit
  end

  # Clenshaw algorithm
  tau = (2.0*t-Ta-Tb)/(Tb-Ta)  

  f1 = zeros(3)
  f2 = zeros(3) 
  old_f1=zeros(3)

  for i in N:-1:2
    old_f1 = copy(f1) 
    f1 = 2.0*tau*f1-f2+[Cx[i],Cy[i],Cz[i]] 
    f2 = copy(old_f1)
  end
  
  r_Moon.=tau*f1-f2+[Cx[1],Cy[1],Cz[1]]


  return nothing

end


MoonCheb (generic function with 1 method)

In [12]:
N_Step=8
Step=0.5

Mjd0 = Mjd(2006,3,14,0,0,0.0)
#Mjd0= 53808

r_Moon=zeros(3)
for i in 0:N_Step
    Mjd_TT = Mjd0 + i*Step
    MoonCheb(r_Moon,Mjd_TT, N_coeff,t1,t2,Cx_moon,Cy_moon,Cz_moon)
    println("Mjd_TT=",Mjd_TT, ", Moon=",r_Moon)
end

Mjd_TT=53808.0, Moon=[-386976.7831388396, 106369.21888706, 61240.44196184228]
Mjd_TT=53808.5, Moon=[-397199.2259141454, 70584.99563744929, 41725.65005983121]
Mjd_TT=53809.0, Moon=[-403002.3311435525, 34008.82556762028, 21741.254760131054]
Mjd_TT=53809.5, Moon=[-404301.8729067812, -2946.2438238462273, 1512.8683942005046]
Mjd_TT=53810.0, Moon=[-401058.6500841973, -39859.47969148816, -18729.305439323947]
Mjd_TT=53810.5, Moon=[-393279.7517060793, -76305.82367242243, -38752.305199214316]
Mjd_TT=53811.0, Moon=[-381019.56286902056, -111859.42309552713, -58322.34096169204]
Mjd_TT=53811.5, Moon=[-364380.4911275337, -146097.3111886013, -77206.80806903348]
Mjd_TT=53812.0, Moon=[-343513.4033978065, -178603.2167648899, -95176.37420462182]


### Moon position from low precision analytical theory

In [15]:
Mjd0 = Mjd(2006,3,14,0,0,0.0)

53808.0

In [16]:
N_Step=8
Step=0.5

Mjd0 = Mjd(2006,3,14,0,0,0.0)
#Mjd0= 53808

r_Moon=zeros(3)
for i in 0:N_Step
    Mjd_TT = Mjd0 + i*Step
    Moon!(r_Moon,Mjd_TT)
    println("Mjd_TT=",Mjd_TT, ", Moon=",r_Moon/1000)
end

Mjd_TT=53808.0, Moon=[-387370.19743008976, 106087.1045084343, 61146.65942815451]
Mjd_TT=53808.5, Moon=[-397579.6514271391, 70056.34550905536, 41497.32203526625]
Mjd_TT=53809.0, Moon=[-403305.9337819239, 33232.15328633493, 21375.00795249281]
Mjd_TT=53809.5, Moon=[-404464.3950889143, -3946.427239904606, 1019.8849055775855]
Mjd_TT=53810.0, Moon=[-401024.30346881854, -41034.66237915785, -19324.15442275841]
Mjd_TT=53810.5, Moon=[-393008.87846317474, -77588.25711396812, -39412.91794047623]
Mjd_TT=53811.0, Moon=[-380494.07873646566, -113169.04245615566, -59005.11709005821]
Mjd_TT=53811.5, Moon=[-363606.43575266027, -147349.93521810143, -77865.17757645308]
Mjd_TT=53812.0, Moon=[-342520.32286604383, -179719.00525674288, -95765.55488178808]
